# Getting Started with Biomass CCI Data

### 1. Import Necessary Packages

In this section, we import the required Python packages to work with ESA Climate Change Initiative (CCI) data. Most notably, we use the [ESA Climate Toolbox](https://esa-climate-toolbox.readthedocs.io/en/latest/) which simplifies access, manipulation, and visualization of CCI datasets in Python.

These packages allow us to:
- Access satellite-based climate data records from ESA.
- Handle geospatial and temporal dimensions efficiently.
- Visualize data with intuitive plotting tools.

📚 For a broader introduction to the toolbox and how to install it, visit:  
🔗 [ESA CCI Climate Toolbox Quick Start](https://esa-climate-toolbox.readthedocs.io/en/latest/quick_start.html#jupyter-notebooks)  
🔗 [ESA Climate Data Toolbox Website](https://climate.esa.int/en/data/toolbox/)

In [1]:
from xcube.core.store import new_data_store
from esa_climate_toolbox.core import get_op
from esa_climate_toolbox.core import list_ecv_datasets
from esa_climate_toolbox.core import get_store
from esa_climate_toolbox.core import list_datasets
from esa_climate_toolbox.ops import plot
import xarray as xr
import matplotlib.pyplot as plt
import warnings
%matplotlib inline

# Suppress FutureWarnings related to xarray subclassing
warnings.filterwarnings("ignore", category=FutureWarning, message=".*xarray subclass VectorDataCube.*")

<frozen abc>:106: FutureWarning: xarray subclass VectorDataCube should explicitly define __slots__


### Step 2: Connect to the ESA CCI Data Store

The ESA Climate Toolbox provides direct access to the **ESA Climate Data Store**, which hosts harmonized satellite-based climate data records produced under the **ESA Climate Change Initiative (CCI)**.

In this step, we establish a connection to the data store so we can browse and open datasets. This connection allows us to access data **without having to download files manually** — a convenient way to explore and analyze large geospatial datasets in cloud-friendly formats such as **Zarr** or **Kerchunk**.

The data store includes a wide range of essential climate variables (ECVs), such as aerosols, land surface temperature, sea level, and soil moisture.

📘 Learn more about available datasets:  
🔗 [ESA Climate Data Toolbox – Quick Start Guide](https://esa-climate-toolbox.readthedocs.io/en/latest/quick_start.html#data-access)

In [2]:
cci_store = new_data_store("esa-cci")
# List all available data sets of an ECV
# list_ecv_datasets("Biomass")

### Step 3: Define the Dataset ID  
To work with a specific ESA CCI dataset, we need to specify its **dataset ID**. This unique identifier tells the toolbox which variable and product we want to access.

In this example, we are using a dataset that provides **Above-Ground Biomass (AGB)**, derived from a combination of multiple sensors and platforms. AGB is a key Essential Climate Variable (ECV) used to estimate the amount of carbon stored in vegetation, supporting studies related to the global carbon cycle, deforestation, and land use change.

For other ESA CCI projects (e.g., Land Cover, Soil Moisture, Sea Surface Temperature), simply replace the dataset ID accordingly.

📘 A full list of dataset IDs can be retrieved from the store or found in the [ESA CCI Climate Toolbox documentation](https://esa-climate-toolbox.readthedocs.io/en/latest/data_access.html#available-datasets).

In [3]:
data_id = 'esacci.BIOMASS.yr.L4.AGB.multi-sensor.multi-platform.MERGED.5-0.10000m'

This product provides annual AGB estimates at 10000m resolution, covering the period from 2018 to 2021.

### Step 4: Describe Dataset (Check Available Variables and Metadata)

Before loading the full dataset, it’s helpful to **inspect the metadata** to understand its structure. This includes:

- Available variables (e.g., AOD, uncertainty estimates)
- Temporal and spatial coverage
- Data format and structure

This step ensures we know what the dataset contains and how to work with it. It also helps confirm that the variable we want to plot or analyze is actually included.

🛠️ Tip: You can use the description to verify variable names, dimensions (e.g., `lat`, `lon`, `time`), and time coverage.

📘 More on dataset structure:  
🔗 [ESA Climate Toolbox – Data Access](https://esa-climate-toolbox.readthedocs.io/en/latest/data_access.html)

In [4]:
cci_store.describe_data(data_id)

/cmsaf/cmsaf-ops3/stkothe/KE/ect/lib/python3.12/site-packages/xcube_cci/cciodp.py:1960: CciOdpWarning: Variable "crs" has no fill value, cannot set one. For parts where no data is available you will see random values. This is usually the case when data is missing for a time step.
  warnings.warn(f'Variable "{fixed_key}" has no fill value, '


### Step 5: Check Open Parameters for the Dataset

Before opening the dataset, we can inspect which parameters are supported by the **Zarr opener** (e.g., `time_range`, `bbox`, `variable_names`). This step helps ensure that we pass valid arguments when loading data and avoid errors.

The command below lists all expected input parameters and their allowed values for the selected dataset.

In [5]:
cci_store.get_open_data_params_schema(data_id=data_id, opener_id='dataset:zarr:cciodp')

### Step 6: Define Region, Time Range, and Variables of Interest

Before opening the dataset, we define a few key parameters:

- **Time range**: the date(s) we want to load
- **Variables**: which data variable(s) to retrieve
- *(Optional)* **Bounding box**: spatial region of interest — here we skip it to load the global dataset

In [6]:
variables = ['agb']  # Variable to retrieve
start_date = '2018-01-01'    # Start and end date (same for a single timestep)
end_date = '2021-12-31'

### Step 7: Open the Dataset

Now we open the dataset using the selected parameters.  
The ESA Climate Toolbox will download only the necessary data (e.g., variable and time range).
You can always adjust the time range or variables to explore different slices of the dataset.

In [7]:
biomass_ds = cci_store.open_data(
    data_id=data_id,
    variable_names=variables,
    time_range=[start_date, end_date]
)

### Step 8: Display Dataset Structure

We print a summary of the opened dataset to inspect its structure, dimensions, variables, and metadata.  
This helps verify that the data was loaded correctly and shows what is available for analysis and visualization.
This step is useful to understand what the dataset contains before working with it further.

In [8]:
print(biomass_ds.time.values)

['2018-07-02T12:00:00.000000000' '2019-07-02T12:00:00.000000000'
 '2020-07-02T00:00:00.000000000' '2021-07-02T12:00:00.000000000']


In [9]:
print("\nOpened Dataset:\n", biomass_ds)


Opened Dataset:
 <xarray.Dataset> Size: 104MB
Dimensions:    (time: 4, lat: 1800, lon: 3600, nv: 2, bnds: 2)
Coordinates:
  * lat        (lat) float64 14kB 89.95 89.85 89.75 ... -89.75 -89.85 -89.95
    lat_bnds   (lat, nv) float64 29kB dask.array<chunksize=(1800, 2), meta=np.ndarray>
  * lon        (lon) float64 29kB -179.9 -179.8 -179.8 ... 179.8 179.9 180.0
    lon_bnds   (lon, nv) float64 58kB dask.array<chunksize=(3600, 2), meta=np.ndarray>
  * time       (time) datetime64[ns] 32B 2018-07-02T12:00:00 ... 2021-07-02T1...
    time_bnds  (time, bnds) datetime64[ns] 64B dask.array<chunksize=(4, 2), meta=np.ndarray>
Dimensions without coordinates: nv, bnds
Data variables:
    agb        (time, lat, lon) float32 104MB dask.array<chunksize=(4, 450, 450), meta=np.ndarray>
    crs        <U0 0B ...
Attributes:
    Conventions:             CF-1.7
    title:                   esacci.BIOMASS.yr.L4.AGB.multi-sensor.multi-plat...
    date_created:            2025-04-10T08:36:49.261269
    proc

### Step 9: Visualize Results

We now create a simple map plot of the selected variable.  
This allows us to explore the spatial patterns of the data — in this case, the Aerosol Optical Depth (AOD) for the selected day.
For more interactive and advanced visualizations, check out the [ESA Climate Toolbox](https://climate.esa.int/en/data/toolbox/) or the [Toolbox documentation](https://esa-climate-toolbox.readthedocs.io/en/latest/quick_start.html#jupyter-notebooks).

In [10]:
for i, t in enumerate(biomass_ds.time.values):
    print(f"Index {i}: {t}")

Index 0: 2018-07-02T12:00:00.000000000
Index 1: 2019-07-02T12:00:00.000000000
Index 2: 2020-07-02T00:00:00.000000000
Index 3: 2021-07-02T12:00:00.000000000


In [11]:
agb = biomass_ds["agb"].isel(time=3)

# Subset to central Europe to avoid full chunk fetch
agb_subset = agb.sel(
    lat=slice(55, 45),   # south to north
    lon=slice(5, 15)     # west to east
)
print("\nOpened Dataset:\n", agb_subset)


Opened Dataset:
 <xarray.DataArray 'agb' (lat: 100, lon: 100)> Size: 40kB
dask.array<getitem, shape=(100, 100), dtype=float32, chunksize=(100, 100), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 800B 54.95 54.85 54.75 54.65 ... 45.25 45.15 45.05
  * lon      (lon) float64 800B 5.05 5.15 5.25 5.35 ... 14.65 14.75 14.85 14.95
    time     datetime64[ns] 8B 2021-07-02T12:00:00
Attributes: (12/14)
    long_name:         Above-ground biomass
    grid_mapping:      crs
    units:             Mg/ha
    valid_min:         0.0
    valid_max:         10000.0
    orig_data_type:    int16
    ...                ...
    shape:             [4, 1800, 3600]
    chunk_sizes:       [8, 450, 450]
    file_chunk_sizes:  [8, 450, 450]
    data_type:         int16
    dimensions:        ['time', 'lat', 'lon']
    file_dimensions:   ['time', 'lat', 'lon']


In [12]:
import matplotlib.pyplot as plt

# Select subset *before* selecting time to avoid messing with chunk index tracking
agb_all_time = biomass_ds["agb"]

# Subset to Central Europe (small region to keep memory demands low)
agb_ce = agb_all_time.sel(
    lat=slice(55, 45),   # north to south (note: lat usually descends)
    lon=slice(5, 15)     # west to east
)

# Now select the time index (which is known to be 3 for July 2021)
agb_ce_t3 = agb_ce.isel(time=3)

# Now plot
plt.figure(figsize=(8, 6))
agb_ce_t3.plot(
    cmap="YlGn",
    robust=True,
    cbar_kwargs={"label": "Above Ground Biomass (Mg/ha)"}
)
plt.title("AGB – Central Europe – July 2021")
plt.tight_layout()
plt.show()

IndexError: list index out of range

<Figure size 800x600 with 0 Axes>